In [1]:
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value
import csv


In [2]:


dmus = ["BANGALORE DAIRY BRANCH","BANGALORE MICO ADUGODI","Banglaore, Madiwala","BTM LAYOUT BRANCH, BTM 2nd Stage","VIVEKNAGAR","Gandhi Bazar","HANUMANTHNAGAR","ISRO LAYOUT","No.38, 5th Cross, Malleshwaram","Yelahanka"]
Ips = ['No of Employees','Sum(Operations Cost)','Sum(Salaries, Wages, and Fixed Costs)']
Ops = ['Sum(AdjSales)','Count of clients']



In [3]:
X = {
    i:{
        k: 0 for k in dmus
    } for i in Ips
}

Y = {
    j: {
        k: 0 for k in dmus
    } for j in Ops
}

In [4]:
with open('BankBranchesSyntheticData.csv',newline='') as csvfile:
    df_dict = csv.DictReader(csvfile)
    k=0
    for row in df_dict:
        for i in Ips:
            X[i][dmus[k]] = float(row[i])
        for j in Ops:
            Y[j][dmus[k]] = float(row[j])
        k+=1

# CCR - CRS Model - Constant Return to Scale

In [5]:
def branchEffCCR(dmu):    #CCR
    print("Branch--------------------------",dmu)
    model = LpProblem('CRS_model',LpMinimize)
    theta_r = LpVariable(f'theta_r')
    lambda_k = LpVariable.dicts(f'lambda_k',lowBound=0, indices=dmus)
    model += theta_r
    #Constraints
    for i in Ips:
        model += lpSum([
            lambda_k[k] * X[i][k]
            for k in dmus]) <= theta_r *float(X[i][dmu])

    for j in Ops:
        model += lpSum([
            lambda_k[k] * Y[j][k]
            for k in dmus]) >= float(Y[j][dmu])
        
    model.solve()
    return f'{dmu}：{round(value(model.objective), 3)}\n', value(model.objective)

    


In [6]:
for dmu in dmus:
    a = branchEffCCR(dmu)
    print(a)



Branch-------------------------- BANGALORE DAIRY BRANCH
('BANGALORE DAIRY BRANCH：0.999\n', 0.99852902)
Branch-------------------------- BANGALORE MICO ADUGODI
('BANGALORE MICO ADUGODI：0.997\n', 0.9974387)
Branch-------------------------- Banglaore, Madiwala
('Banglaore, Madiwala：1.0\n', 1.0)
Branch-------------------------- BTM LAYOUT BRANCH, BTM 2nd Stage
('BTM LAYOUT BRANCH, BTM 2nd Stage：0.995\n', 0.99476872)
Branch-------------------------- VIVEKNAGAR
('VIVEKNAGAR：1.0\n', 1.0)
Branch-------------------------- Gandhi Bazar
('Gandhi Bazar：0.117\n', 0.11699054)
Branch-------------------------- HANUMANTHNAGAR
('HANUMANTHNAGAR：0.986\n', 0.98642751)
Branch-------------------------- ISRO LAYOUT
('ISRO LAYOUT：1.0\n', 1.0)
Branch-------------------------- No.38, 5th Cross, Malleshwaram
('No.38, 5th Cross, Malleshwaram：0.999\n', 0.99915073)
Branch-------------------------- Yelahanka
('Yelahanka：0.997\n', 0.99698921)


# BCC - VRS Model - Variable Return to Scale 

In [7]:
def branchEffVRS(dmu):
    model = LpProblem('VRS_model',LpMinimize)
    theta_r = LpVariable(f'theta_r')
    lambda_k = LpVariable.dicts(f'lambda_k', lowBound=0, indices= dmus)
    model += theta_r

    for i in Ips:
        model += lpSum([
            lambda_k[k] * X[i][k]
            for k in dmus]) <= theta_r * float(X[i][dmu])
    for j in Ops:
        model += lpSum([
            lambda_k[k] * Y[j][k]
            for k in dmus]) >= float(Y[j][dmu])
        
    model += lpSum([lambda_k[k] for k in dmus]) ==1
    model.solve()
    return f'{dmu}:{round(value(model.objective), 3)}\n', value(model.objective)


In [8]:
for dmu in dmus:
    print("Branch--------------------------",dmu)
    b = branchEffVRS(dmu)
    print(b)



Branch-------------------------- BANGALORE DAIRY BRANCH
('BANGALORE DAIRY BRANCH:0.999\n', 0.99870308)
Branch-------------------------- BANGALORE MICO ADUGODI
('BANGALORE MICO ADUGODI:0.998\n', 0.99752453)
Branch-------------------------- Banglaore, Madiwala
('Banglaore, Madiwala:1.0\n', 1.0)
Branch-------------------------- BTM LAYOUT BRANCH, BTM 2nd Stage
('BTM LAYOUT BRANCH, BTM 2nd Stage:0.995\n', 0.9948336)
Branch-------------------------- VIVEKNAGAR
('VIVEKNAGAR:1.0\n', 1.0)
Branch-------------------------- Gandhi Bazar
('Gandhi Bazar:0.842\n', 0.84210526)
Branch-------------------------- HANUMANTHNAGAR
('HANUMANTHNAGAR:1.0\n', 1.0)
Branch-------------------------- ISRO LAYOUT
('ISRO LAYOUT:1.0\n', 1.0)
Branch-------------------------- No.38, 5th Cross, Malleshwaram
('No.38, 5th Cross, Malleshwaram:0.999\n', 0.99940454)
Branch-------------------------- Yelahanka
('Yelahanka:0.997\n', 0.99733612)
